In this challenge, you'll use this dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013.

To complete this challenge:

1. Determine the five most common journals and the total articles for each. 
2. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

You will need to do considerable data cleaning in order to extract accurate estimates. You may may want to look into data encoding methods if you get stuck. 

3. For a real bonus round, identify the open access prices paid by subject area.

Remember not to modify the data directly. Instead, write a cleaning script that will load the raw data and whip it into shape. Keep a record of your decisions: well-commented code is a must for recording your data cleaning decision-making progress.

In [1]:
import numpy as np
import pandas as pd
import re
from difflib import SequenceMatcher

In [2]:
df = pd.read_csv("wellcome.csv",encoding='unicode_escape')
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [3]:
df = df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'cost'})


# Parse all Journal titles to strings and print number of unique values
df["Journal title"] = [str(i) for i in df["Journal title"]]
print(len(np.unique(df["Journal title"])))

#np.unique(df["Journal title"])

985


Issues noticed:

1. Repition with different capitalization
2. Repition with different punctuation marks
3. Repition based on trailing spaces and \n chars or chars that end with \xa0
5. Repition based on punctuation translation (ex. and, &)
6. Repition based on trailing punctuation marks
7. Repition based on abbreviation
8. nan value


The array produced 985 values, lets' write a cleaning script and see how many duplicates were consolidated:

In [4]:
# Strip values of surrounding whitespace
df["Journal title"] = df["Journal title"].apply(lambda string: string.strip())
print(len(np.unique(df["Journal title"])))

952


In [5]:
# Remove non-alphanumeric, non space characters
df["Journal title"] = df["Journal title"].apply(lambda value: re.sub('[^a-zA-Z0-9\s]','',value))

print(len(np.unique(df["Journal title"])))


948


In [6]:
# Convert J., Jnl abbreviations to Journal, in beginning and end of values
df["Journal title"] = df["Journal title"].apply(lambda value: re.sub(' (jn?l?.?) ','Journal',value,flags=re.IGNORECASE))
df["Journal title"] = df["Journal title"].apply(lambda value: re.sub('^(jn?l?.?)\s','Journal ',value,flags=re.IGNORECASE))
df["Journal title"] = df["Journal title"].apply(lambda value: re.sub('\s\s',' ',value))

print(len(np.unique(df["Journal title"])))

943


In [7]:
# Function to unify capitalization
def unify_caps(string):
    return " ".join([i.capitalize() for i in string.split(" ")])

df["Journal title"] = df["Journal title"].apply(unify_caps)

print(len(np.unique(df["Journal title"])))


886


In [8]:
# Replace & characters with 'And'
df["Journal title"] = df["Journal title"].str.replace("&","And")

print(len(np.unique(df["Journal title"])))


886


In [9]:
# Function to match similar strings with a percentage
def similar(a, b): 
    return SequenceMatcher(None, a, b).ratio()
# If strings have over a 95% match, choose the one with least length
for i in range(len(df["Journal title"])-1):
    if similar(df["Journal title"][i],df["Journal title"][i+1]) > 0.95:
        if len(df["Journal title"][i]) < len(df["Journal title"][i+1]):
            continue
        df["Journal title"][i] = df["Journal title"][i+1]
        
print(len(np.unique(df["Journal title"])))

879


## 1. Determine the five most common journals and the total articles for each. 


In [17]:
df['Journal title'].value_counts(normalize=True).head(5)

Plos One                           0.089798
Journal Of Biological Chemistry    0.024918
Neuroimage                         0.013634
Nucleic Acids Research             0.012224
Plos Pathogens                     0.011283
Name: Journal title, dtype: float64

In [34]:
df0 = df[['Journal title', 'cost']]
def sbp_2_float(value):
    if value.startswith('£'):
        return float(value[1:])
    else:
        return float(value[:-1])

df0['cost'] =  df0['cost'].apply(sbp_2_float)
mean_df = df0.groupby('Journal title').aggregate(np.mean)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
